# Setup

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean
import random
import numpy as np
from copy import deepcopy
import itertools
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings, get_cell_filepaths

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


### Import settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [2]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

# Transfer annotations from Mark to MAIN

In [5]:
# directory_path = Path(settings_dict['save_dir'])
directory_path_mark = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/CREST_reconstructions/Spine_Density_Annotation/from_mark')#annotated cells from local desktop')#returned from main network folder/')#474111084/')
directory_path_hannah = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/CREST_reconstructions/Spine_Density_Annotation/from_main')


In [6]:
cell_filepaths_mark = get_cell_filepaths(directory_path_mark) # gets filepaths for all cells in a directory"
cell_filepaths_hannah = get_cell_filepaths(directory_path_hannah) # gets filepaths for all cells in a directory"

cells to do:
<!-- ['128737253', -->
 '214581797',
 '299249397',
 '299496636',
 '300474334',
 '301787806',
 '307418797',
 '386117124',
 '386224676',
 '393325331',
 '394115741',
 '472392041']

## open the cell from each location

Only create a neuroglancer instance for the cell that you will be importing annotations into

In [ ]:
id_ = '472392041'

crest_mark = ecrest(settings_dict,filepath = cell_filepaths_mark[id_], launch_viewer=False)
crest_hannah = ecrest(settings_dict,filepath = cell_filepaths_hannah[id_], launch_viewer=True)

### create annotation layers

In [ ]:
# add annotation layers for spine data
crest_hannah.add_endpoint_annotation_layers(['spineD pts','spineD loc'],link=False) # 

In [ ]:
## use this if made a mistake and need to remove an annotation layer
# crest_main.del_endpoint_annotation_layers(['spine_inputs'])

### spineD loc

In [ ]:
# loop through each sphere annotation in spineD loc and add the annotation 
for p in crest_mark.cell_data['end_points']['spineD loc']:
    # print(p)
    try:
        if p[2] == 'annotateSphere':
            c = [p_/i for p_,i in zip(p[0],[16,16,30])]
            r = [p_/i for p_,i in zip(p[1],[16,16,30])]
            # print(p,c,r)
            crest_hannah.add_endpoint_annotation('spineD loc',to_vox=False, center = c, radii = r) # 'center' is in middle of dendrite in voxels
    except:
        print(p)
        

In [ ]:
## You can use this code to check out any annotation locations that are not spheres (maybe it was meant to be but was missing?)
# loc = [232288.0, 246720.0, 17820.0]
# [p_/i for p_,i in zip(loc,[16,16,30])]

### spineD pts

In [ ]:
# for the point annotations, they should be able to be added all at once
crest_hannah.cell_data['end_points']['spineD pts']= crest_mark.cell_data['end_points']['spineD pts']
crest_hannah.load_annotation_layer_points() # need to actually load them from cell_data

### Save if annotations loaded properly

In [ ]:
crest_hannah.save_cell_graph(directory_path = directory_path_hannah)

### MAKE CORRECTIONS, THEN SAVE AGAIN

This includes adding segments to reconstructions if the apical dendritic tree is incomplete

In [ ]:
crest_hannah.save_cell_graph(directory_path = directory_path_hannah)

# Analysis of the data

This is just a draft of some basic functions to get it started... needs to be tailored to actual questions and split by cell type, etc

In [ ]:
cell_filepaths_hannah = get_cell_filepaths(directory_path_hannah) # gets filepaths for all cells in a directory"

In [ ]:
cellids = list(cell_filepaths.keys())

In [7]:
id_ = '472392041'
cell = ecrest(settings_dict,filepath = cell_filepaths_mark[id_], launch_viewer=True)

updating viewer status message: Current Base Segment Counts: unknown: 6771, axon: 137, basal dendrite: 26, apical dendrite: 350, dendrite: 0, multiple: 731


In [ ]:
anno_loc = 'spineD loc'
anno_pts = 'spineD pts'
vx_sizes = [16, 16, 30]
'''assumes that the annotation is a point annotation stored in the list as ([x,y,z,segment_id],'annotatePoint')
                previous ot Jan 25 2024, it was just [x,y,z,segment_id]'''

data = []

for id_ in cellids:
    # id_ = cellids[0]
    
    cell = ecrest(settings_dict,filepath = cell_filepaths[id_])
    
    ctype = cell.get_ctype("manual")
    

    
    for loc_ in cell.cell_data['end_points'][anno_loc]:
        x,y,z = [p/1000 for p in loc_[0]]             
    
        d = [np.linalg.norm(np.array(pt_[0]) - np.array(loc_[0])) for pt_ in cell.cell_data['end_points'][anno_pts] if (np.linalg.norm(np.array(pt_[0]) - np.array(loc_[0])))<5100]
        
        data.append({'id': id_, 'cell_type': ctype, 'x': x, 'y': y, 'z': z, 'n': len(d)})#, ignore_index=True)

df = pd.DataFrame(data)

In [ ]:
ax = sns.scatterplot(data=df,x='x',y='y',size='n',hue='id',sizes=(2, 200),legend=False)
# ax.set_aspect('equal', adjustable='datalim')
ax.set_ylim(0,300)
ax.set_xlim(0,500)
# for y_ in [2365*16, 4344*16, 8432*16, 11138*16, 13021*16, 15045*16, 15700*16]:
#     ax.axhline(y=y_/1000,color = 'black',linestyle='--')
ax.invert_yaxis()


In [ ]:
sns.kdeplot(data=df,x='n',hue='id',cut=0)

In [ ]:
ax = sns.scatterplot(data=df,x='n',y='y',hue='id')#,legend=False)
# ax.set_aspect('equal', adjustable='datalim') 
ax.invert_yaxis()
for y_ in [2365*16, 4344*16, 8432*16, 11138*16, 13021*16, 15045*16, 15700*16]:
    ax.axhline(y=y_/1000,color = 'black',linestyle='--')

In [ ]:
edges = array([2365, 4344, 8432, 11138, 13021, 15045, 15700])*16/1000

for i,v in enumerate(edges[0:-1]):
    w = edges[i+1]
    u = df[(df['y']>v) & (df['y']<w)]['n'].mean()
    print(f'{u} spines per 5microns from {v} to {w} microns in molec layer')

In [ ]:
df['y']